##Modelado con Scikit-learn

In [ ]:
import pandas as pd
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

# Cargar y preparar datos
url = "https://tauusbmededu-my.sharepoint.com/personal/jeferson_vega231_tau_usbmed_edu_co/_layouts/15/download.aspx?share=EVPG5kJ7esxAhgCt3ftLrGcBT9hZs4KHxOkUwFialqFV-Q"

df = pd.read_csv(url)

object_columns = df.select_dtypes(include='object').columns
for col in object_columns:
    df[col] = df[col].astype(str).str.lower()
    df[col] = df[col].astype('category').cat.codes
# Dividir en características (X) y etiquetas (y)
X = df.drop('target', axis=1)
y = df['target']

# Escalar los datos
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Modelo ANN
mlp = MLPClassifier(hidden_layer_sizes=(100,50), activation='tanh', max_iter=300, random_state=42)
mlp.fit(X_train, y_train)

# Predicción y evaluación
y_pred = mlp.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       159
           1       1.00      1.00      1.00       149

    accuracy                           1.00       308
   macro avg       1.00      1.00      1.00       308
weighted avg       1.00      1.00      1.00       308



## Modelo Keras

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Cargar datos
digits = df
X = df.drop("target", axis=1)
y = df["target"]

# Escalado
scaler = StandardScaler()
X = scaler.fit_transform(X)

# One-hot encoding de las etiquetas
y_cat = to_categorical(y)

# División
X_train, X_test, y_train_cat, y_test_cat = train_test_split(X, y_cat, test_size=0.3, random_state=42)
y_train, y_test = train_test_split(y, test_size=0.3, random_state=42)

# Modelo Keras
model = Sequential([
    Dense(100, activation='relu', input_shape=(X.shape[1],)),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=30, batch_size=32, verbose=0)

# Evaluación
y_pred_probs = model.predict(X_test)
y_pred = np.argmax(y_pred_probs, axis=1)

print(classification_report(y_test, y_pred))


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
              precision    recall  f1-score   support

           0       0.52      1.00      0.68       159
           1       0.00      0.00      0.00       149

    accuracy                           0.52       308
   macro avg       0.26      0.50      0.34       308
weighted avg       0.27      0.52      0.35       308



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


##Modelado con PyTorch


In [ ]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset

url = "https://tauusbmededu-my.sharepoint.com/personal/jeferson_vega231_tau_usbmed_edu_co/_layouts/15/download.aspx?share=EVPG5kJ7esxAhgCt3ftLrGcBT9hZs4KHxOkUwFialqFV-Q"

df = pd.read_csv(url)

object_columns = df.select_dtypes(include='object').columns
for col in object_columns:
    df[col] = df[col].astype(str).str.lower()
    df[col] = df[col].astype('category').cat.codes
# Dividir en características (X) y etiquetas (y)
X = df.drop('target', axis=1)
y = df['target']

# Escalado
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Conversión a tensores
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_train_tensor = torch.FloatTensor(X_train)
X_test_tensor = torch.FloatTensor(X_test)
y_train_tensor = torch.LongTensor(y_train.values)

y_test_tensor = torch.LongTensor(y_test.values)

# Dataset y DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Modelo MLP
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(13, 100)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Entrenamiento
for epoch in range(30):
    for inputs, labels in train_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
# Mostrar la pérdida por época
    print(f"Epoch [{epoch+1}/30], Loss: {loss.item():.4f}")
# Evaluación
with torch.no_grad():
    outputs = model(X_test_tensor)
    _, y_pred = torch.max(outputs, 1)

print(classification_report(y_test_tensor, y_pred))


Epoch [1/30], Loss: 1.4547
Epoch [2/30], Loss: 0.8390
Epoch [3/30], Loss: 0.6253
Epoch [4/30], Loss: 0.4811
Epoch [5/30], Loss: 0.4207
Epoch [6/30], Loss: 0.2040
Epoch [7/30], Loss: 0.4135
Epoch [8/30], Loss: 0.2928
Epoch [9/30], Loss: 0.3814
Epoch [10/30], Loss: 0.2618
Epoch [11/30], Loss: 0.4252
Epoch [12/30], Loss: 0.6712
Epoch [13/30], Loss: 0.5682
Epoch [14/30], Loss: 0.2609
Epoch [15/30], Loss: 0.3068
Epoch [16/30], Loss: 0.4031
Epoch [17/30], Loss: 0.1402
Epoch [18/30], Loss: 0.4563
Epoch [19/30], Loss: 0.0608
Epoch [20/30], Loss: 0.1920
Epoch [21/30], Loss: 0.1536
Epoch [22/30], Loss: 0.2553
Epoch [23/30], Loss: 0.1878
Epoch [24/30], Loss: 0.2179
Epoch [25/30], Loss: 0.3224
Epoch [26/30], Loss: 0.3294
Epoch [27/30], Loss: 0.2860
Epoch [28/30], Loss: 0.1825
Epoch [29/30], Loss: 0.1580
Epoch [30/30], Loss: 0.4138
              precision    recall  f1-score   support

           0       0.89      0.86      0.87       159
           1       0.85      0.89      0.87       149

    a